Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as SNS

from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, \
                            plot_confusion_matrix, plot_roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier

import xgboost

In [2]:
# read in cleaned data file
df = pd.read_csv('final_data.csv', index_col=0)
df.head()

,amount_tsh,funder,gps_height,installer,basin,region_code,district_code,public_meeting,scheme_management,permit,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,status_group,age
0,6000.0,Other,1390,Other,Lake Nyasa,11,5,True,VWC,False,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional,23
1,0.0,Other,1399,Other,Lake Victoria,20,2,Unknown,Other,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,12
2,25.0,Other,686,Other,Pangani,21,4,True,VWC,True,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,functional,13
3,0.0,Unicef,263,Other,Ruvuma / Southern Coast,90,63,True,VWC,True,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,non functional,36
4,0.0,Other,0,Other,Lake Victoria,18,1,True,Unknown,True,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional,22


In [3]:
# Create our predictors and target
X = df.drop('status_group', axis=1)
y = df.status_group

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
X_train.head()

,amount_tsh,funder,gps_height,installer,basin,region_code,district_code,public_meeting,scheme_management,permit,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,age
4442,0.0,Other,947,Other,Ruvuma / Southern Coast,10,2,True,VWC,True,gravity,vwc,never pay,soft,enough,river,communal standpipe,35
11884,0.0,Other,227,Other,Ruvuma / Southern Coast,8,3,True,VWC,False,gravity,vwc,never pay,unknown,dry,rainwater harvesting,communal standpipe,20
38674,4000.0,Other,1980,Other,Rufiji,11,7,True,VWC,True,gravity,vwc,pay annually,soft,enough,river,communal standpipe,19
34584,500.0,Unicef,1471,DWE,Rufiji,11,4,True,WUA,True,gravity,wua,pay monthly,soft,enough,river,communal standpipe,44
58645,0.0,Unknown,0,Unknown,Rufiji,12,7,True,VWC,True,gravity,vwc,pay monthly,soft,seasonal,river,communal standpipe,22


In [6]:
# Confirm that out training data set is 3x larger than the testing dataset
print(X_train.shape, X_test.shape)

(44271, 18) (14758, 18)


# Modeling

### Dummy Model

First, we will create a dummy model that always predicts the most frequent class. In this case, our dummy model will predict that all of the water wells are functional.

In [7]:
# Instantiate our dummy regressor object
dummy = DummyClassifier(strategy='most_frequent')

In [8]:
# Fit the model on our training data
dummy.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [9]:
# Inspect the accuracy of our initial dummy model
dummy.score(X_train, y_train)

0.5457071220437758

Our dummy model accuracy of 54.6% reflects the distribution of our target. Thus, if we always predict that a water well is functional, we will achieve 54.6% accuracy.

### Set Up Column Transformer

In [10]:
# Create our sub pipelines for numeric and categorial data
subpipe_num = Pipeline(steps=[('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse=False, 
                                                    handle_unknown='ignore'))])

In [11]:
# Include our sub pipelines in a column transformer to OHE our categorial variables and scale our numerical variables.
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [0,2,17]), 
                                     ('subpipe_cat', subpipe_cat, [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16])])

### Create Pipeline for Simple Models

In [12]:
logreg_pipe = Pipeline(steps=[('CT', CT), 
                              ('logreg', LogisticRegression(random_state=42))])

knn_pipe = Pipeline(steps=[('CT', CT), 
                              ('knn', KNeighborsClassifier())])

dtc_pipe = Pipeline(steps=[('CT', CT), 
                              ('dtc', DecisionTreeClassifier(random_state=42))])


In [13]:
# Fit each pipeline to our training data
logreg_pipe.fit(X_train, y_train)
knn_pipe.fit(X_train, y_train)
dtc_pipe.fit(X_train, y_train)

/Users/zacharypollatsek/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [0, 2, 17]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [1, 3, 4, 5, 6, 7, 8, 9, 10,
                                                   11, 12, 13, 14, 15, 16])])),
                ('dtc', DecisionTreeClassifier(random_state=42))])

In [14]:
# Get initial accuracy scores for each our our simple classifier models
logreg_score = logreg_pipe.score(X_train, y_train)
knn_score = knn_pipe.score(X_train, y_train)
dtc_score = dtc_pipe.score(X_train, y_train)

In [19]:
# Print out initial accuracy scores
print(f'Logistic Regression Accuracy: {round(logreg_score * 100, 2)}%')
print(f'KNearestNeighbors Accuracy: {round(knn_score * 100, 2)}%')
print(f'DecisionTree Accuracy: {round(dtc_score * 100, 2)}%')

Logistic Regression Accuracy: 74.68%
KNearestNeighbors Accuracy: 82.61%
DecisionTree Accuracy: 94.4%


Our initial numbers appear to be relatively good. We expect our decision tree classifier to be overfit given the high accuracy score. We will dive deeper into more complex models below to maximize our accuracy and recall. 

### Random Forest

In [23]:
# Create our random forest pipeline to incorporate column transformer
rfc_pipe = Pipeline(steps=[('CT', CT),
                          ('rfc', RandomForestClassifier(random_state=42))])

In [24]:
# Fit our random forest classifier to the training data
rfc_pipe.fit(X_train, y_train)

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  [0, 2, 17]),
                                                 ('subpipe_cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [1, 3, 4, 5, 6, 7, 8, 9, 10,
                                                   11, 12, 13, 14, 15, 16])])),
                ('rfc', RandomForestClassifier(random_state=42))])

In [25]:
rfc_pipe.score(X_train, y_train)

0.9439813873641888

In [26]:
cross_val_score(estimator=rfc_pipe, X=X_train, y=y_train, cv=5)

array([0.7796725 , 0.78032528, 0.7798735 , 0.78134177, 0.77716286])

Given that our cross validation scores were significantly lower than the accuracy score from our single random forest classifier, we can say that this model is overfit to the training data. We will perform a grid search in order to optimize the hyper parameters of this classifier.

In [27]:
# create our grid for the grid search parameters
params = {'rfc__n_estimators': [50, 100, 150],
          'rfc__min_samples_split': [2, 10, 50],
          'rfc__max_depth': [5, 10, 15]}

# set up GridSearchCV object
grid_rfc = GridSearchCV(rfc_pipe, param_grid=params, cv=5)

In [28]:
# fit our grid object for random forests to the training data
grid_rfc.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         [0, 2,
                                                                          17]),
                                                                        ('subpipe_cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                            

In [29]:
grid_rfc.best_params_

{'rfc__max_depth': 15, 'rfc__min_samples_split': 2, 'rfc__n_estimators': 100}

In [50]:
grid_rfc.best_score_

0.7832668202933669

From our initial grid search, we attained an accuracy of 78.3% using a max_depth of 15, a min_samples_split of 2, and an n_estimators of 100. 

In [30]:
pd.DataFrame(grid_rfc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__max_depth,param_rfc__min_samples_split,param_rfc__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.301719,0.011407,0.088801,0.001377,5,2,50,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.713608,0.722611,0.724644,0.726790,0.707703,0.719071,0.007242,26
1,2.387787,0.008930,0.120239,0.002130,5,2,100,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.717335,0.724418,0.727129,0.727468,0.707590,0.720788,0.007538,22
2,3.468344,0.014413,0.152590,0.003440,5,2,150,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.719142,0.723402,0.725548,0.727468,0.709736,0.721059,0.006302,20
3,1.307158,0.018561,0.088471,0.002444,5,10,50,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.714512,0.723176,0.724080,0.727242,0.707477,0.719297,0.007262,25
4,2.397931,0.012328,0.120913,0.002938,5,10,100,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.717109,0.724531,0.727468,0.728145,0.708267,0.721104,0.007517,19
5,3.510307,0.037176,0.151582,0.003910,5,10,150,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.719255,0.723515,0.725096,0.727807,0.709623,0.721059,0.006355,21
6,1.304617,0.006007,0.087595,0.002325,5,50,50,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.713721,0.721369,0.724080,0.726790,0.704992,0.718190,0.007911,27
7,2.385140,0.004705,0.121148,0.002780,5,50,100,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.718577,0.722950,0.725548,0.727581,0.707590,0.720449,0.007099,24
8,3.472620,0.015733,0.153408,0.005454,5,50,150,"{'rfc__max_depth': 5, 'rfc__min_samples_split'...",0.719819,0.722498,0.725548,0.727468,0.708493,0.720765,0.006668,23
9,1.840314,0.006085,0.100718,0.002525,10,2,50,"{'rfc__max_depth': 10, 'rfc__min_samples_split...",0.747374,0.750960,0.755817,0.756833,0.741586,0.750514,0.005618,14


Based on our initial grid search results, we can infer that a larger max depth is needed to improve accuracy. 

In [31]:
# create our grid for the grid search parameters
params2 = {'rfc__n_estimators': [50, 100, 150],
           'rfc__max_depth': [15, 25, 50]}

# set up GridSearchCV object
grid_rfc2 = GridSearchCV(rfc_pipe, param_grid=params2, cv=5)

In [32]:
# fit our grid object with parameter updates for random forests to the training data
grid_rfc2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         [0, 2,
                                                                          17]),
                                                                        ('subpipe_cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                            

In [33]:
grid_rfc2.best_params_

{'rfc__max_depth': 25, 'rfc__n_estimators': 150}

In [34]:
grid_rfc2.best_score_

0.7897721682958518

From our second grid search, we attained an accuracy of 79.0% using a max_depth of 25 and an n_estimators of 150. 

In [35]:
pd.DataFrame(grid_rfc2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__max_depth,param_rfc__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.346743,0.038142,0.128413,0.000396,15,50,"{'rfc__max_depth': 15, 'rfc__n_estimators': 50}",0.775833,0.783714,0.784504,0.786311,0.776711,0.781415,0.004292,6
1,4.451738,0.036551,0.199298,0.002288,15,100,"{'rfc__max_depth': 15, 'rfc__n_estimators': 100}",0.777866,0.785747,0.785182,0.790490,0.777050,0.783267,0.005095,4
2,6.524388,0.033046,0.269549,0.002430,15,150,"{'rfc__max_depth': 15, 'rfc__n_estimators': 150}",0.778543,0.785521,0.785521,0.788344,0.777276,0.783041,0.004333,5
3,2.853596,0.019079,0.159120,0.002047,25,50,"{'rfc__max_depth': 25, 'rfc__n_estimators': 50}",0.784980,0.790038,0.788344,0.792862,0.783488,0.787943,0.003388,3
4,5.488861,0.023299,0.260454,0.004628,25,100,"{'rfc__max_depth': 25, 'rfc__n_estimators': 100}",0.785658,0.791845,0.790603,0.793878,0.784504,0.789298,0.003617,2
5,8.154395,0.038337,0.362912,0.004045,25,150,"{'rfc__max_depth': 25, 'rfc__n_estimators': 150}",0.786110,0.793314,0.791620,0.793991,0.783827,0.789772,0.004063,1
6,3.064216,0.014483,0.172049,0.002432,50,50,"{'rfc__max_depth': 50, 'rfc__n_estimators': 50}",0.778656,0.780325,0.779083,0.782358,0.776485,0.779382,0.001937,8
7,5.992171,0.020339,0.284298,0.000437,50,100,"{'rfc__max_depth': 50, 'rfc__n_estimators': 100}",0.780350,0.780099,0.778292,0.781229,0.775695,0.779133,0.001967,9
8,8.838696,0.057840,0.400096,0.002814,50,150,"{'rfc__max_depth': 50, 'rfc__n_estimators': 150}",0.779447,0.780890,0.779309,0.781229,0.777615,0.779698,0.001290,7


Based on our new grid search, we can see that a maximum depth greater than 15 but less than 50 is ideal. We will also try running the grid search with a higher number of trees in the random forest to improve accuracy.

In [36]:
# create our grid for the grid search parameters
params3 = {'rfc__n_estimators': [100, 150, 200],
           'rfc__max_depth': [20, 25, 30]}

# set up GridSearchCV object
grid_rfc3 = GridSearchCV(rfc_pipe, param_grid=params3, cv=5)

In [37]:
# Fit the grid object to our training data with updated grid params for n_estimators and max_depth
grid_rfc3.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         [0, 2,
                                                                          17]),
                                                                        ('subpipe_cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                            

In [39]:
grid_rfc3.best_params_

{'rfc__max_depth': 20, 'rfc__n_estimators': 150}

In [40]:
grid_rfc3.best_score_

0.7932732831246889

From our third grid search, we attained an accuracy of 79.3% using a max_depth of 20 and an n_estimators of 150. 

In [41]:
pd.DataFrame(grid_rfc3.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__max_depth,param_rfc__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.114668,0.100837,0.237210,0.003148,20,100,"{'rfc__max_depth': 20, 'rfc__n_estimators': 100}",0.789046,0.795686,0.796476,0.797267,0.786989,0.793093,0.004224,3
1,7.448913,0.050085,0.330273,0.003286,20,150,"{'rfc__max_depth': 20, 'rfc__n_estimators': 150}",0.791756,0.795008,0.795121,0.797041,0.787441,0.793273,0.003375,1
2,10.470498,0.729096,0.429496,0.017237,20,200,"{'rfc__max_depth': 20, 'rfc__n_estimators': 200}",0.792321,0.795686,0.794895,0.795686,0.787441,0.793206,0.003136,2
3,5.773903,0.526419,0.265385,0.003499,25,100,"{'rfc__max_depth': 25, 'rfc__n_estimators': 100}",0.785658,0.791845,0.790603,0.793878,0.784504,0.789298,0.003617,6
4,8.407452,0.210512,0.376479,0.020376,25,150,"{'rfc__max_depth': 25, 'rfc__n_estimators': 150}",0.786110,0.793314,0.791620,0.793991,0.783827,0.789772,0.004063,5
5,11.816350,0.881208,0.479841,0.006052,25,200,"{'rfc__max_depth': 25, 'rfc__n_estimators': 200}",0.785884,0.794104,0.792410,0.794556,0.783601,0.790111,0.004500,4
6,5.945509,0.322162,0.280625,0.002535,30,100,"{'rfc__max_depth': 30, 'rfc__n_estimators': 100}",0.780237,0.784504,0.784504,0.785408,0.778970,0.782725,0.002601,9
7,9.577850,0.544449,0.414980,0.022117,30,150,"{'rfc__max_depth': 30, 'rfc__n_estimators': 150}",0.783060,0.785747,0.784956,0.785069,0.778292,0.783425,0.002717,8
8,13.304384,1.550077,0.502973,0.011423,30,200,"{'rfc__max_depth': 30, 'rfc__n_estimators': 200}",0.781705,0.785747,0.785634,0.786085,0.779648,0.783764,0.002608,7


In [44]:
# create our grid for the grid search parameters
params4 = {'rfc__n_estimators': [100, 150],
           'rfc__max_depth': [18, 20, 22],
           'rfc__max_features': ['auto', 'sqrt', 0.2]}

# set up GridSearchCV object
grid_rfc4 = GridSearchCV(rfc_pipe, param_grid=params4, cv=5)

In [45]:
grid_rfc4.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('ss',
                                                                                          StandardScaler())]),
                                                                         [0, 2,
                                                                          17]),
                                                                        ('subpipe_cat',
                                                                         Pipeline(steps=[('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                            

In [47]:
grid_rfc4.best_params_

{'rfc__max_depth': 20, 'rfc__max_features': 0.2, 'rfc__n_estimators': 150}

In [48]:
grid_rfc4.best_score_

0.7934991544239145

From our fourth grid search, we attained an accuracy of 79.4% using a max_depth of 20, a max_features of 0.2, and an n_estimators of 150. 

In [49]:
pd.DataFrame(grid_rfc4.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__max_depth,param_rfc__max_features,param_rfc__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.854097,0.079085,0.222644,0.000962,18,auto,100,"{'rfc__max_depth': 18, 'rfc__max_features': 'a...",0.788142,0.791845,0.792749,0.796476,0.786311,0.791105,0.003574,17
1,7.026312,0.030303,0.305097,0.001928,18,auto,150,"{'rfc__max_depth': 18, 'rfc__max_features': 'a...",0.787126,0.793540,0.793878,0.795799,0.786424,0.791353,0.003823,15
2,4.772689,0.075151,0.224029,0.002694,18,sqrt,100,"{'rfc__max_depth': 18, 'rfc__max_features': 's...",0.788142,0.791845,0.792749,0.796476,0.786311,0.791105,0.003574,17
3,7.014761,0.042045,0.306500,0.002449,18,sqrt,150,"{'rfc__max_depth': 18, 'rfc__max_features': 's...",0.787126,0.793540,0.793878,0.795799,0.786424,0.791353,0.003823,15
4,9.448839,0.041567,0.219691,0.001682,18,0.2,100,"{'rfc__max_depth': 18, 'rfc__max_features': 0....",0.790966,0.794104,0.795008,0.797606,0.787780,0.793093,0.003400,7
5,13.961023,0.081917,0.303138,0.002018,18,0.2,150,"{'rfc__max_depth': 18, 'rfc__max_features': 0....",0.791417,0.795121,0.797041,0.794556,0.789022,0.793431,0.002851,2
6,4.942688,0.019941,0.234814,0.001883,20,auto,100,"{'rfc__max_depth': 20, 'rfc__max_features': 'a...",0.789046,0.795686,0.796476,0.797267,0.786989,0.793093,0.004224,5
7,7.298178,0.026983,0.334742,0.007693,20,auto,150,"{'rfc__max_depth': 20, 'rfc__max_features': 'a...",0.791756,0.795008,0.795121,0.797041,0.787441,0.793273,0.003375,3
8,4.965089,0.039019,0.238054,0.002290,20,sqrt,100,"{'rfc__max_depth': 20, 'rfc__max_features': 's...",0.789046,0.795686,0.796476,0.797267,0.786989,0.793093,0.004224,5
9,7.324271,0.083424,0.328797,0.001753,20,sqrt,150,"{'rfc__max_depth': 20, 'rfc__max_features': 's...",0.791756,0.795008,0.795121,0.797041,0.787441,0.793273,0.003375,3


### XGBoost
